In [3]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_6.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((299, 299))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve_2"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 0.5897054076194763
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.3622729182243347
Tp_S_NNN_S_B_nat00029_nat00029_00645.jpeg - Manipulated: 0.0007944858516566455
Au_pla_30253.jpg - Authentic: 0.7020174860954285
Au_pla_20040.jpg - Manipulated: 0.3970148265361786
Tp_S_NNN_S_N_arc00076_arc00076_01052.jpeg - Manipulated: 0.0005627577193081379
Tp_S_NNN_S_N_cha00015_cha00015_00844.jpeg - Manipulated: 0.001974137732759118
Au_pla_30247.jpg - Authentic: 0.9264823198318481
Au_pla_20054.jpg - Manipulated: 0.30815279483795166
Tp_S_NNN_S_N_arc00027_arc00027_01095.jpeg - Manipulated: 0.0039358483627438545
Au_pla_30521.jpg - Authentic: 0.8673259019851685
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Manipulated: 0.0009861050639301538
Tp_S_NNN_S_N_art20027_art20027_01833.jpeg - Manipulated: 0.2888837456703186
Au_pla_30509.jpg - Authentic: 0.9891013503074646
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Manipulated: 0.009794613346457481
Au_pla_20068.jpg - Manipulat

Au_sec_00057.jpg - Authentic: 0.8420126438140869
Au_sec_30194.jpg - Authentic: 0.9817325472831726
Tp_S_NNN_S_N_ind00063_ind00063_00909.jpeg - Manipulated: 0.00244097295217216
Tp_S_NNN_S_N_arc10101_arc10101_11168.jpg - Manipulated: 0.01259835995733738
Au_sec_00043.jpg - Authentic: 0.9011461138725281
Au_sec_30180.jpg - Authentic: 0.9995176792144775
Tp_S_NNN_S_N_sec20034_sec20034_01624.jpeg - Manipulated: 0.08471778780221939
Tp_S_NNN_S_N_art10010_art10010_20093.jpg - Manipulated: 0.4026716351509094
Au_nat_30389.jpg - Authentic: 0.9784051775932312
Au_pla_30469.jpg - Manipulated: 0.18404611945152283
Au_pla_30482.jpg - Manipulated: 0.03459986299276352
Tp_S_NNN_S_B_nat20002_nat20002_02202.jpeg - Authentic: 0.5398394465446472
Au_nat_30404.jpg - Authentic: 0.9612103700637817
Tp_S_NNN_S_B_sec20080_sec20080_01670.jpeg - Manipulated: 0.009541140869259834
Au_nat_30362.jpg - Authentic: 0.9446466565132141
Tp_S_NNN_S_N_pla00031_pla00031_10985.jpg - Manipulated: 0.005078174639493227
Tp_S_NRN_S_B_nat200

Au_pla_30050.jpg - Authentic: 0.9982309341430664
Tp_S_NNN_S_N_cha00049_cha00049_00369.jpeg - Manipulated: 0.02301260456442833
Tp_S_NNN_S_O_ani00021_ani00021_20026.jpg - Manipulated: 0.0716007724404335
Au_pla_30736.jpg - Authentic: 0.9987310767173767
Tp_S_NNN_S_N_ani00071_ani00071_00827.jpeg - Manipulated: 0.006565322168171406
Au_sec_30036.jpg - Manipulated: 0.23460958898067474
Au_nat_30559.jpg - Authentic: 0.5279244184494019
Tp_S_NNN_S_N_cha10181_cha10181_12296.jpg - Manipulated: 0.4137149155139923
Tp_S_NNN_S_N_nat20055_nat20055_02250.jpeg - Manipulated: 0.4399136006832123
Au_nat_20038.jpg - Manipulated: 0.1027911826968193
Au_sec_30022.jpg - Authentic: 0.9997109174728394
Tp_S_NRN_S_N_cha10130_cha10130_12186.jpg - Manipulated: 0.05624747276306152
Tp_S_NNN_S_B_ind00056_ind00056_01345.jpeg - Manipulated: 0.012922929599881172
Au_nat_20010.jpg - Manipulated: 0.38836389780044556
Au_nat_30203.jpg - Authentic: 0.9964287281036377
Tp_S_NNN_S_B_nat20001_nat20001_01505.jpeg - Manipulated: 0.437676

Tp_S_NNN_S_B_ind00076_ind00076_01362.jpeg - Manipulated: 0.00492862006649375
Tp_S_NNN_S_N_art00063_art00063_01237.jpeg - Manipulated: 0.013849534094333649
Tp_S_NNN_S_N_nat20047_nat20047_01551.jpeg - Manipulated: 0.12203849107027054
Au_pla_30126.jpg - Authentic: 0.9998465776443481
Au_nat_30188.jpg - Authentic: 0.9998567700386047
Au_sec_30381.jpg - Authentic: 0.9999662041664124
Tp_S_NRN_S_N_arc20065_arc20065_01705.jpeg - Manipulated: 0.05350062623620033
Tp_S_NNN_S_N_ani10222_ani10222_12395.jpg - Manipulated: 0.0819302424788475
Tp_S_NRN_S_N_cha20042_cha20042_02041.jpeg - Manipulated: 0.40649139881134033
Au_pla_30668.jpg - Authentic: 0.9983593821525574
Au_txt_00015.jpg - Manipulated: 0.09206711500883102
Tp_S_NNN_S_N_nat20053_nat20053_01557.jpeg - Manipulated: 0.4105832576751709
Tp_S_NNN_S_N_sec00077_sec00077_00752.jpeg - Manipulated: 0.0024436565581709146
Au_txt_00001.jpg - Authentic: 0.7557127475738525
Au_sec_30395.jpg - Manipulated: 0.010781028307974339
Tp_S_NNN_S_B_sec10001_sec10001_200

Tp_S_NRN_S_N_pla00016_pla00016_10961.jpg - Manipulated: 0.001901933690533042
Au_pla_00033.jpg - Authentic: 0.9929801821708679
Au_nat_30176.jpg - Authentic: 0.6595761775970459
Tp_S_NNN_S_N_ind20012_ind20012_02271.jpeg - Manipulated: 0.4383055865764618
Au_nat_30610.jpg - Authentic: 0.9999463558197021
Au_pla_30696.jpg - Authentic: 0.9996344447135925
Tp_S_NNN_S_N_nat20026_nat20026_01530.jpeg - Manipulated: 0.02372780442237854
Tp_S_NNN_S_N_ind20050_ind20050_02296.jpeg - Manipulated: 0.10033319145441055
Au_sec_30419.jpg - Manipulated: 0.14859822392463684
Tp_S_NNN_S_N_ind00029_ind00029_00896.jpeg - Manipulated: 0.001060830312781036
Au_sec_30394.jpg - Authentic: 0.9921116828918457
Tp_S_NNN_S_N_nat20050_nat20050_02246.jpeg - Manipulated: 0.38720938563346863
Au_sec_30380.jpg - Authentic: 0.9730935096740723
Tp_S_NNN_S_N_nat10107_nat10107_11348.jpg - Manipulated: 0.02534993551671505
Au_nat_30189.jpg - Authentic: 0.6353873610496521
Au_txt_00014.jpg - Authentic: 1.0
Au_pla_30669.jpg - Authentic: 0.9

...

Au_pla_30066.jpg - Authentic: 0.9999970197677612
Tp_S_NNN_S_N_ani00006_ani00006_00833.jpeg - Manipulated: 0.0032882923260331154
Au_pla_30700.jpg - Authentic: 0.9689329862594604
Tp_S_NNN_S_N_ani00081_ani00081_00819.jpeg - Manipulated: 0.002679842058569193
Tp_S_NNN_S_N_sec00020_sec00020_00761.jpeg - Manipulated: 0.00236043194308877
Au_nat_30209.jpg - Authentic: 0.9986355304718018
Au_sec_30014.jpg - Authentic: 0.9991292953491211
Au_pla_20088.jpg - Manipulated: 0.40204811096191406
Tp_S_NNN_S_B_nat00019_nat00019_20032.jpg - Manipulated: 0.001056573004461825
Au_nat_20026.jpg - Manipulated: 0.013558718375861645
Au_nat_30235.jpg - Authentic: 0.8896707892417908
Tp_S_NNN_S_B_ind00078_ind00078_01364.jpeg - Manipulated: 0.0024665547534823418
Au_nat_30553.jpg - Manipulated: 0.3343896269798279
Au_nat_30547.jpg - Authentic: 0.9966478943824768
Au_sec_30028.jpg - Authentic: 0.8163066506385803
Tp_S_NRN_S_O_cha00035_cha00067_11734.jpg - Manipulated: 0.004540166351944208
Au_nat_20032.jpg - Manipulated: 0.

Au_nat_30153.jpg - Authentic: 0.9992902278900146
Au_nat_30147.jpg - Manipulated: 0.35886698961257935
Tp_S_NNN_S_N_cha00066_cha00066_10321.jpeg - Manipulated: 0.001277889939956367
Au_pla_00002.jpg - Authentic: 0.9902123808860779
Tp_S_NRN_S_N_txt00083_txt00083_01289.jpeg - Manipulated: 0.4100993871688843
Au_sec_30428.jpg - Authentic: 0.5748703479766846
Tp_S_NRD_S_N_cha10154_cha10154_12266.jpg - Manipulated: 0.006044871639460325
Au_nat_30621.jpg - Authentic: 0.982084333896637
Tp_S_NNN_S_N_arc20028_arc20028_02169.jpeg - Manipulated: 0.40663057565689087
Tp_S_NRN_S_N_pla20088_pla20088_01988.jpeg - Manipulated: 0.42132672667503357
Au_pla_30328.jpg - Authentic: 0.9960805177688599
Au_sec_00064.jpg - Authentic: 0.9997113347053528
Au_sec_00070.jpg - Authentic: 0.8500370979309082
Tp_S_NRD_S_N_ani00078_ani00078_00822.jpeg - Manipulated: 0.006036969367414713
Au_pla_30314.jpg - Manipulated: 0.28414595127105713
Au_sec_00058.jpg - Authentic: 0.997826099395752
Au_nat_30392.jpg - Authentic: 0.64711332321

Au_sec_30575.jpg - Authentic: 0.9945237040519714
Au_sec_30213.jpg - Authentic: 0.999983012676239
Tp_S_NRN_S_N_txt00009_txt00009_01267.jpeg - Manipulated: 0.0015048820059746504
Au_pla_30077.jpg - Authentic: 0.940112292766571
Au_pla_30711.jpg - Authentic: 0.9966636896133423
Tp_S_NNN_S_B_ind00064_ind00064_01352.jpeg - Manipulated: 0.03854558244347572
Au_pla_30705.jpg - Authentic: 0.9856740236282349
Au_pla_30063.jpg - Manipulated: 0.29163968563079834
Tp_S_NRN_S_N_art20037_art20037_01843.jpeg - Manipulated: 0.4114368259906769
Au_sec_10114.jpg - Manipulated: 0.24104304611682892
Tp_S_NNN_S_N_ind00078_ind00078_10662.jpg - Manipulated: 0.007784469053149223
Au_pla_30739.jpg - Authentic: 0.9954365491867065
Tp_S_NRN_S_N_ani00033_ani00033_00153.jpeg - Manipulated: 0.11656516790390015
Tp_S_NRD_S_B_nat00038_nat00038_00957.jpeg - Manipulated: 0.017185259610414505
Tp_S_NNN_S_N_sec20074_sec20074_01664.jpeg - Manipulated: 0.047201190143823624
Tp_S_NNN_S_N_sec00036_sec00036_00737.jpeg - Manipulated: 0.010

Tp_S_NNN_S_N_art20012_art20012_01818.jpeg - Manipulated: 0.03593641519546509
Au_pla_30101.jpg - Manipulated: 0.4401572644710541
Au_nat_30187.jpg - Authentic: 0.9978961944580078
Au_nat_30193.jpg - Authentic: 0.6290784478187561
Au_pla_30115.jpg - Authentic: 0.9999997019767761
Au_pla_30673.jpg - Authentic: 0.8479353785514832
Au_nat_10163.jpeg - Manipulated: 0.4431951940059662
Tp_S_NNN_S_N_cha10114_cha10114_12175.jpg - Manipulated: 0.3740530014038086
Tp_S_NRN_S_N_nat20051_nat20051_01555.jpeg - Manipulated: 0.13674640655517578
Au_sec_30417.jpg - Manipulated: 0.43619757890701294
Au_pla_30698.jpg - Authentic: 0.6366335153579712
Tp_S_NRN_S_N_cha00081_cha00081_10190.jpeg - Manipulated: 0.0018475922988727689
Au_sec_30371.jpg - Authentic: 0.6586958169937134
Au_nat_30178.jpg - Authentic: 0.9999968409538269
Tp_S_NNN_S_B_sec00083_sec00083_11286.jpg - Manipulated: 0.0032097294460982084
Tp_S_NNN_S_N_sec20026_sec20026_02452.jpeg - Manipulated: 0.008127430453896523
Au_pla_00029.jpg - Authentic: 0.804495

Au_pla_30699.jpg - Authentic: 0.9999561309814453
Au_sec_30416.jpg - Manipulated: 0.0752824917435646
Au_nat_30179.jpg - Authentic: 0.9999131560325623
Au_sec_30370.jpg - Authentic: 0.6767892837524414
Tp_S_NRN_S_B_txt00022_txt00022_20089.jpg - Manipulated: 0.0068451291881501675
Au_pla_30114.jpg - Manipulated: 0.4084472358226776
Tp_S_NNN_S_N_pla00044_pla00044_01154.jpeg - Manipulated: 0.0006970555405132473
Tp_S_NRN_S_B_sec20014_sec20014_01449.jpeg - Manipulated: 0.4273644983768463
Au_nat_30192.jpg - Authentic: 0.8649604320526123
Tp_S_NNN_S_N_ind20062_ind20062_01803.jpeg - Manipulated: 0.007498207502067089
Au_pla_30672.jpg - Authentic: 0.9999827146530151
Au_pla_30666.jpg - Authentic: 0.9998480081558228
Tp_S_NRN_S_N_cha10196_cha10196_12304.jpg - Manipulated: 0.006850032135844231
Au_nat_30186.jpg - Authentic: 0.576102077960968
Au_pla_30100.jpg - Authentic: 0.9958429336547852
Au_pla_30128.jpg - Authentic: 0.9500575065612793
Au_txt_00033.jpg - Authentic: 0.9872488975524902
Tp_S_NNN_S_N_art20054